In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from itertools import combinations
import datetime
from scipy.stats import entropy
from collections import Counter

# --- Agent クラス および 補助関数 (省略、変更なし) ---
class Agent:
    def __init__(self, bitN_meaning, bitN_form, m2s, s2m, i):
        self.bitN_meaning = bitN_meaning
        self.bitN_form = bitN_form
        self.m2s = m2s
        self.s2m = s2m
        self.m2m = nn.Sequential(m2s, s2m)
        self.num = i
        
def create_agent(bitN_meaning, bitN_form, nodeN, i):
    m2s = nn.Sequential(
        nn.Linear(bitN_meaning, nodeN), nn.Sigmoid(),
        nn.Linear(nodeN, bitN_form), nn.Sigmoid()
    )
    s2m = nn.Sequential(
        nn.Linear(bitN_form, nodeN), nn.Sigmoid(),
        nn.Linear(nodeN, bitN_meaning), nn.Sigmoid()
    )
    return Agent(bitN_meaning, bitN_form, m2s, s2m, i)

def int2bin(bitN, value):
    return [int(x) for x in f"{value:0{bitN}b}"]

def generate_structured_meaning_space(N_A, N_P, N_R, N_F=1):
    bitN_meaning = N_A + N_P + N_R + N_F
    all_meanings = []
    meaning_pairs = []
    predicates_tensors = [torch.tensor(int2bin(N_R, i), dtype=torch.float32) for i in range(2 ** N_R)]
    agents_tensors = [torch.tensor(int2bin(N_A, i), dtype=torch.float32) for i in range(2 ** N_A)]
    patients_tensors = [torch.tensor(int2bin(N_P, i), dtype=torch.float32) for i in range(2 ** N_P)]
    foci = [torch.tensor([0.0], dtype=torch.float32), torch.tensor([1.0], dtype=torch.float32)]

    for R in predicates_tensors:
        for A in agents_tensors:
            for P in patients_tensors:
                situation_parts = [A, P, R]
                M_act = torch.cat(situation_parts + [foci[0]])
                M_pass = torch.cat(situation_parts + [foci[1]])
                all_meanings.append(M_act)
                all_meanings.append(M_pass)
                meaning_pairs.append((M_act, M_pass))
    return all_meanings, meaning_pairs

def gen_supervised_data(tutor, all_meanings):
    T = []
    for meaning in all_meanings:
        signal = tutor.m2s(meaning.unsqueeze(0)).detach().round().squeeze(0)
        T.append((meaning.numpy(), signal.numpy()))
    return T

def gen_unsupervised_data(all_meanings, A_size):
    U = []
    for _ in range(A_size):
        meaning = random.choice(all_meanings)
        U.append(meaning.numpy())
    return U

def train_combined(agent, tutor, A_size, B_size, all_meanings, epochs, alpha=5.0):
    optimiser_m2s = torch.optim.SGD(agent.m2s.parameters(), lr=5.0)
    optimiser_s2m = torch.optim.SGD(agent.s2m.parameters(), lr=5.0)
    optimiser_m2m = torch.optim.SGD(list(agent.m2s.parameters()) + list(agent.s2m.parameters()), lr=5.0)
    loss_function = nn.MSELoss(reduction='none')
    T = gen_supervised_data(tutor, all_meanings)
    A = gen_unsupervised_data(all_meanings, A_size)
    N_F = 1
    N_SITUATION = agent.bitN_meaning - N_F

    for epoch in range(epochs):
        B1 = [random.choice(T) for _ in range(B_size)]
        B2 = B1.copy()
        random.shuffle(B2)

        for i in range(B_size):
            # M -> S
            optimiser_m2s.zero_grad()
            m2s_meaning, m2s_signal = B1[i]
            m2s_meaning = torch.tensor(m2s_meaning, dtype=torch.float32).unsqueeze(0)
            m2s_signal = torch.tensor(m2s_signal, dtype=torch.float32).unsqueeze(0)
            pred_m2s = agent.m2s(m2s_meaning)
            loss_m2s = loss_function(pred_m2s, m2s_signal).mean()
            loss_m2s.backward()
            optimiser_m2s.step()

            # S -> M
            optimiser_s2m.zero_grad()
            s2m_meaning, s2m_signal = B2[i]
            s2m_signal = torch.tensor(s2m_signal, dtype=torch.float32).unsqueeze(0)
            s2m_meaning = torch.tensor(s2m_meaning, dtype=torch.float32).unsqueeze(0)
            pred_s2m = agent.s2m(s2m_signal)
            loss_s2m = loss_function(pred_s2m, s2m_meaning).mean()
            loss_s2m.backward()
            optimiser_s2m.step()

            # Autoencoder (M -> S -> M)
            meanings_u = [random.choice(A) for _ in range(20)]
            for meaning in meanings_u:
                optimiser_m2m.zero_grad()
                auto_m = torch.tensor(meaning, dtype=torch.float32).unsqueeze(0)
                pred_m2m = agent.m2m(auto_m)
                
                loss_elements = loss_function(pred_m2m, auto_m)
                loss_situation = loss_elements[:, :N_SITUATION]
                loss_focus = loss_elements[:, N_SITUATION:]
                weighted_loss_focus = alpha * loss_focus
                loss_auto = torch.cat((loss_situation, weighted_loss_focus), dim=1).mean()
                
                loss_auto.backward()
                optimiser_m2m.step()
                
    return T
                
def iterated_learning(generations=20, N_A=2, N_P=2, N_R=2, N_F=1, bitN_form=5, nodeN=8, A_size=75, B_size=75, epochs=20, alpha=5.0):
    
    bitN_meaning = N_A + N_P + N_R + N_F
    tutor = create_agent(bitN_meaning, bitN_form, nodeN, 1)

    stability_scores = []
    expressivity_scores = []
    compositionality_scores = []
    alternation_scores = []
    topsim_scores = [] # ★ TopSim スコアのリスト
    posdis_scores = []
    
    all_meanings, meaning_pairs = generate_structured_meaning_space(N_A, N_P, N_R, N_F)
    
    all_meaning_signal_pairs = []

    for gen in range(1, generations + 1):
        pupil = create_agent(bitN_meaning, bitN_form, nodeN, gen)
        current_T = train_combined(pupil, tutor, A_size, B_size, all_meanings, epochs, alpha=alpha)
        all_meaning_signal_pairs.append(current_T)

        stability_scores.append(stability(tutor, pupil, all_meanings))
        expressivity_scores.append(expressivity(pupil, all_meanings))
        compositionality_scores.append(compositionality(pupil, all_meanings))
        alternation_scores.append(alternation(pupil, all_meanings, meaning_pairs))
        topsim_scores.append(TopSim(pupil, all_meanings)) # ★ TopSim スコアを計算・追加
        posdis_scores.append(PosDis(pupil, all_meanings))

        tutor = pupil

    return (np.array(stability_scores), 
            np.array(expressivity_scores), 
            np.array(compositionality_scores), 
            np.array(alternation_scores), 
            np.array(topsim_scores), 
            np.array(posdis_scores), 
            all_meaning_signal_pairs)

# --- スコア計算関数 (省略、変更なし) ---
def stability(tutor, pupil, all_meanings):
    tutor.m2s.eval()
    pupil.s2m.eval()
    matches = 0
    total_meanings = len(all_meanings)
    with torch.no_grad():
        for meaning in all_meanings:
            m = meaning.clone().detach().float().unsqueeze(0)
            tutor_m2s_sig = tutor.m2s(m)
            pupil.s2m.eval()
            pupil_s2m_mn = pupil.s2m(tutor_m2s_sig)
            original_arr = meaning.numpy() > 0.5
            decoded_arr = pupil_s2m_mn.squeeze(0).numpy() > 0.5
            if np.array_equal(original_arr, decoded_arr):
                matches += 1
    return matches / total_meanings

def expressivity(agent, all_meanings):
    agent.m2s.eval()
    unique_signals = set()
    with torch.no_grad():
        for meaning in all_meanings:
            signal = tuple(agent.m2s(meaning).round().squeeze(0).numpy().astype(int))
            unique_signals.add(signal)
    return len(unique_signals) / (2 ** agent.bitN_form)

def calculate_entropy(p):
    if p <= 0 or p >= 1:
        return 0.0
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def compositionality(agent, all_meanings):
    agent.m2s.eval()
    n_m = agent.bitN_meaning
    n_f = agent.bitN_form
    num_messages = len(all_meanings)
    meaning_matrix = np.zeros((n_m, num_messages), dtype=int)
    signal_matrix = np.zeros((n_f, num_messages), dtype=int)

    cnt = 0
    with torch.no_grad():
        for m in all_meanings:
            s = agent.m2s(m.unsqueeze(0)).detach().round().squeeze(0)
            meaning_matrix[:, cnt] = m.numpy()
            signal_matrix[:, cnt] = s.numpy()
            cnt += 1

    fact_min_entropies = np.zeros(n_m)
    fact_best_word = np.zeros(n_m, dtype=int)

    for i in range(n_m):
        min_entropy = np.inf
        best_j = -1
        for j in range(n_f):
            p = np.sum(meaning_matrix[i, :] * signal_matrix[j, :]) / (num_messages / 2)
            h_ij = calculate_entropy(p)

            if h_ij < min_entropy:
                min_entropy = h_ij
                best_j = j
        fact_min_entropies[i] = min_entropy
        fact_best_word[i] = best_j

    adjusted_entropies = fact_min_entropies.copy()
    for j in range(n_f):
        facts_using_j = np.where(fact_best_word == j)[0]
        if len(facts_using_j) > 1:
            best_fact = facts_using_j[np.argmin(fact_min_entropies[facts_using_j])]

            for idx in facts_using_j:
                if idx != best_fact:
                    adjusted_entropies[idx] = 1.0

    average_adjusted_entropy = np.mean(adjusted_entropies)
    return 1 - average_adjusted_entropy

def alternation(agent, all_meanings, meaning_pairs):
    agent.m2s.eval()
    matches = 0
    total_pairs = len(meaning_pairs)
    
    with torch.no_grad():
        for M_act, M_pass in meaning_pairs:
            S_act = agent.m2s(M_act.unsqueeze(0)).round().squeeze(0).numpy().astype(int)
            S_pass = agent.m2s(M_pass.unsqueeze(0)).round().squeeze(0).numpy().astype(int)
            
            if not np.array_equal(S_act, S_pass):
                matches += 1
                
    return matches / total_pairs


def count_meaning_distance_ability(v1, v2):
    distance = np.sum(v1 != v2)
    return distance

def count_meaning_distance_process(meaning_vectors):
    meaning_distance_list = []
    pairs = combinations(meaning_vectors, 2)
    for v1, v2 in pairs:
        distance = count_meaning_distance_ability(v1, v2)
        meaning_distance_list.append(distance)
    return meaning_distance_list

def count_form_distance_ability(v1, v2):
    count = np.sum(v1 != v2)
    return count

def count_form_distance_process(form_vectors):
    form_distance_list = []
    pairs = combinations(form_vectors, 2)
    for v1, v2 in pairs:
        distance = count_form_distance_ability(v1, v2)
        form_distance_list.append(distance)
    return form_distance_list

# ★★★ TopSim 関数 ★★★
def TopSim(agent, all_meanings):
    agent.m2s.eval()
    meaning_vectors = []
    form_vectors = []
    
    with torch.no_grad(): # 勾配計算を停止
        for meaning_tensor in all_meanings:
            meaning_array = meaning_tensor.numpy()
            meaning_vectors.append(meaning_array)
            
            signal_tensor = agent.m2s(meaning_tensor.unsqueeze(0)).round().squeeze(0)
            form_array = signal_tensor.numpy()
            form_vectors.append(form_array)
    
    if len(meaning_vectors) < 2:
        return np.nan
        
    meaning_distance_list = count_meaning_distance_process(meaning_vectors)
    form_distance_list = count_form_distance_process(form_vectors)
    
    if len(meaning_vectors) < 2:
        return np.nan

    TopSim_value, TopSim_p_value = pearsonr(meaning_distance_list, form_distance_list)
    
    # 構成性スコアとして絶対値を返す (通常、TopSimの相関係数は正の値が構成的と解釈される)
    return TopSim_value

### posdis
def calculate_probabilities(data_vector):
    counts = Counter(data_vector)
    total = len(data_vector)

    return {val: count / total for val, count in counts.items()}

def calculate_joint_probabilities(var1, var2): # 同時確率P(X, Y)
    joint_counts = Counter(zip(var1, var2))
    total = len(var1)

    return {pair: count / total for pair, count in joint_counts.items()}

def H(data_vector): # エントロピー計算 (log2を使用)
    probabilities = list(calculate_probabilities(data_vector).values())
    
    return entropy(probabilities, base=2)

def I(var1, var2): # 相互情報量 # I(X; Y) = H(X) + H(Y) - H(X, Y)
    joint_probabilities = list(calculate_joint_probabilities(var1, var2).values())
    H_joint = entropy(joint_probabilities, base=2)
    
    return H(var1) + H(var2) - H_joint



# def PosDis(agent, all_meanings):
    
#     agent.m2s.eval()
#     meaning_vectors = []
#     form_vectors = []
    
#     with torch.no_grad(): # 勾配計算を停止
#         for meaning_tensor in all_meanings:
#             meaning_array = meaning_tensor.numpy()
#             meaning_vectors.append(meaning_array)
            
#             signal_tensor = agent.m2s(meaning_tensor.unsqueeze(0)).round().squeeze(0)
#             form_array = signal_tensor.numpy()
#             form_vectors.append(form_array)
    

#     N, MESSAGE_LEN = form_vectors.shape
#     _, ATTRIBUTES_DIM = meaning_vectors.shape
    
#     posdis_scores = []
    
#     for j in range(MESSAGE_LEN): # 各メッセージ_jのエントロピー
#         s_j = all_messages[:, j]
#         H_s_j = H(s_j)
#         if H_s_j == 0:
#             continue
            
#         I_scores = []
#         for i in range(ATTRIBUTES_DIM): # 各属性_iのエントロピー
#             a_i = all_attributes[:, i]
            
#             I_s_j_a_i = I(s_j, a_i) # メッセージ_j と 属性_i の相互情報量
#             I_scores.append((I_s_j_a_i, i))
            
#         # 3. a_j1 と a_j2 を決定し、情報ギャップを計算
#         if not I_scores:
#             continue
            
#         I_scores.sort(key=lambda x: x[0], reverse=True)  # 相互情報量の降順でソート
        
#         I_aj1 = I_scores[0][0] # 最大の情報量
#         I_aj2 = I_scores[1][0] # 2番目に大きい
#         information_gap = I_aj1 - I_aj2 # I(s_j; a_j1) - I(s_j; a_j2) の情報ギャップ
        
#         # posdis の項を計算し、リストに追加
#         posdis_term = information_gap / H_s_j
#         posdis_scores.append(posdis_term)

#     # 5. すべての項の平均を最終スコアとする
#     if not posdis_scores:
#         return 0.0 # 有効な位置がない場合
    
#     PosDis_value = np.mean(posdis_scores)
        
#     return PosDis_value

# ... (TopSim関数群と補助関数は省略) ...

def PosDis(agent, all_meanings):
    
    agent.m2s.eval()
    meaning_vectors_list = [] # 一時的なリスト名に変更
    form_vectors_list = []    # 一時的なリスト名に変更
    
    with torch.no_grad():
        for meaning_tensor in all_meanings:
            meaning_array = meaning_tensor.numpy()
            meaning_vectors_list.append(meaning_array)
            
            signal_tensor = agent.m2s(meaning_tensor.unsqueeze(0)).round().squeeze(0)
            form_array = signal_tensor.numpy()
            form_vectors_list.append(form_array)
    
    # ★★★ 修正箇所：リストを一つのNumPy行列にスタック ★★★
    all_messages = np.stack(form_vectors_list, axis=0)      # 形式行列 (N x M_LEN)
    all_attributes = np.stack(meaning_vectors_list, axis=0) # 意味行列 (N x ATT_DIM)

    N, MESSAGE_LEN = all_messages.shape
    _, ATTRIBUTES_DIM = all_attributes.shape
    
    posdis_scores = []
    
    # 修正: ループ内で all_messages, all_attributes を使用
    
    for j in range(MESSAGE_LEN): # 各メッセージビット j
        s_j = all_messages[:, j] # j列目 (形式ビット j の値リスト)
        H_s_j = H(s_j)
        if H_s_j == 0:
            continue
            
        I_scores = []
        for i in range(ATTRIBUTES_DIM): # 各属性ビット i
            a_i = all_attributes[:, i] # i列目 (意味ビット i の値リスト)
            
            I_s_j_a_i = I(s_j, a_i) # メッセージ j と 属性 i の相互情報量
            I_scores.append((I_s_j_a_i, i))
            
        # 3. a_j1 と a_j2 を決定し、情報ギャップを計算
        if not I_scores or len(I_scores) < 2:
            continue
            
        I_scores.sort(key=lambda x: x[0], reverse=True)
        
        I_aj1 = I_scores[0][0]
        I_aj2 = I_scores[1][0]
        information_gap = I_aj1 - I_aj2
        
        posdis_term = information_gap / H_s_j
        posdis_scores.append(posdis_term)

    if not posdis_scores:
        return 0.0
        
    PosDis_value = np.mean(posdis_scores)
        
    return PosDis_value



# --- ★ 修正・新設: 個別試行の結果をプロットする関数 (rep_i/figures/ に格納) ---
def plot_individual_results(score_array, score_name, generations, save_path):
    # rep_i/figures には、その試行の結果のみを格納する (単独線)
    gens = np.arange(1, generations + 1)
    
    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        
    color = {'stability': 'purple', 'expressivity': 'blue', 'compositionality': 'orange', 'alternation': 'red', 'topsim': 'green', 'posdis': 'grey'}.get(score_name, 'black')

    fig = plt.figure(figsize=(6, 4))
    # 単独の試行結果のみをプロット
    plt.plot(gens, score_array, color=color, linewidth=3)
    
    plt.xlabel("Generations", fontsize=13)
    plt.ylabel(score_name, fontsize=14)
    plt.ylim(0.00, 1.00) 

    if save_path is not None:
        # ファイル名は平均と分けるため、そのまま (repフォルダ内なのでユニーク)
        file_path = os.path.join(save_path, f"{score_name}.png")
        plt.savefig(file_path, dpi=300)
    
    plt.close(fig)


# --- ★ 修正: 平均結果をプロットする関数 (average_figures/ に格納) ---
def plot_average_results(all_scores_by_rep, score_name, generations, save_path):
    # average_figures には、全ての個別線と平均線を重ねたグラフを格納する
    gens = np.arange(1, generations + 1)
    
    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
    
    color = {'stability': 'purple', 'expressivity': 'blue', 'compositionality': 'orange', 'alternation': 'red', 'topsim': 'green', 'posdis': 'grey'}.get(score_name, 'black')

    # 全試行のスコアをNumpy配列にスタックし、平均を計算
    stacked_scores = np.stack(all_scores_by_rep, axis=0)
    mean_scores = np.mean(stacked_scores, axis=0)
    
    fig = plt.figure(figsize=(6, 4))
    
    # 1. 全ての個別線をプロット（薄い線）
    for i in range(stacked_scores.shape[0]):
        plt.plot(gens, stacked_scores[i], color=color, alpha=0.2, linewidth=1.5)
        
    # 2. 平均線をプロット（太い線）
    plt.plot(gens, mean_scores, color=color, linewidth=4, label='Average')
    
    plt.xlabel("Generations", fontsize=13)
    plt.ylabel(f"Average {score_name}", fontsize=14)
    plt.ylim(0.00, 1.00) 

    if save_path is not None:
        # ファイル名は Average を示す
        file_path = os.path.join(save_path, f"{score_name}_average.png")
        plt.savefig(file_path, dpi=300)
    
    plt.close(fig)
    
    # Jupyter Notebookで表示するために一度だけplt.show()を呼ぶ
    # Note: 4つの平均グラフが表示される
    plt.show()


# --- ★ save_data_txt 関数を修正 (rep_i/generations/ に保存) ---
def save_data_txt(all_meaning_signal_pairs_by_rep, experiment_root):
    
    for rep_i, gen_data in enumerate(all_meaning_signal_pairs_by_rep):
        
        # 保存パス: rep_i/generations/
        data_save_dir = os.path.join(experiment_root, f"rep_{rep_i}", "generations")
        os.makedirs(data_save_dir, exist_ok=True)
        
        for gen_i, T in enumerate(gen_data):
            filename = f"data_rep_{rep_i}_gen_{gen_i+1}.txt"
            filepath = os.path.join(data_save_dir, filename)
            
            with open(filepath, 'w') as f:
                if T:
                    len_M = len(T[0][0])
                    len_S = len(T[0][1])
                    header = f"# Meaning bits: {len_M}, Signal bits: {len_S}\n"
                    header += "# Format: [M_0 M_1 ... M_{N_M-1}] -> [S_0 S_1 ... S_{N_S-1}]\n"
                    f.write(header)
                
                for meaning_array, signal_array in T:
                    meaning_str = ' '.join(map(str, meaning_array.astype(int)))
                    signal_str = ' '.join(map(str, signal_array.astype(int)))
                    f.write(f"{meaning_str} -> {signal_str}\n")
    print(f"Generation data saved to individual 'rep_i/generations' folders.")


def main():
    generations = 50
    replicates = 5 

    all_stability_scores = []
    all_expressivity_scores = []
    all_compositionality_scores = []
    all_alternation_scores = []
    all_topsim_scores = []
    all_posdis_scores = []
    all_meaning_signal_pairs_by_rep = [] 
    
    N_A, N_P, N_R, N_F, bitN_form = 3, 2, 2, 1, 7
    # N_A, N_P, N_R, N_F, bitN_form = 3, 2, 2, 1, 8
    alpha = 5.0
    
    # --- パスとフォルダ名の動的生成 (Jupyter環境対応) ---
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    bitN_meaning = N_A + N_P + N_R + N_F
    setting_name = f"gen{generations}_m{bitN_meaning}_f{bitN_form}_alpha{int(alpha)}"
    experiment_dir_name = f"exp{timestamp}_{setting_name}"

    current_cwd = os.getcwd() 
    evolang2026_dir = os.path.dirname(current_cwd)
    
    experiment_root = os.path.join(evolang2026_dir, "out", experiment_dir_name)
    
    print(f"Experiment Root: {experiment_root}")
    
    score_names = ['stability', 'expressivity', 'compositionality', 'alternation', 'topsim', 'posdis']
    
    # --- ★ シミュレーションとデータ収集 (ループ内) ---
    for i in range(replicates):
        print(f"\n--- Replicates: {i} ---")
        
        stability, expressivity, compositionality, alternation, topsim, posdis, all_meaning_signal_pairs = iterated_learning(
            generations=generations, N_A=N_A, N_P=N_P, N_R=N_R, N_F=N_F, bitN_form=bitN_form, alpha=alpha)
            
        # 1. スコアとデータ配列を全体リストに追加
        all_stability_scores.append(stability)
        all_expressivity_scores.append(expressivity)
        all_compositionality_scores.append(compositionality)
        all_alternation_scores.append(alternation)
        all_topsim_scores.append(topsim)
        all_posdis_scores.append(posdis)
        all_meaning_signal_pairs_by_rep.append(all_meaning_signal_pairs)

        # 2. 個別プロットを rep_i/figures/ に保存 (単独線)
        rep_folder = os.path.join(experiment_root, f"rep_{i}")
        individual_figures_path = os.path.join(rep_folder, "figures")
        
        # スコアをリストではなく、個別のNumpy配列として渡す
        scores = [stability, expressivity, compositionality, alternation, topsim, posdis]
        for score_array, name in zip(scores, score_names):
            plot_individual_results(score_array, name, generations, individual_figures_path)


    # --- ★ ループ終了後、平均プロットとデータ保存を実行 ---
    
    print("\n--- Saving Final Results ---")
    
    # 1. 平均プロットを average_figures/ に保存 (個別線と平均線を重ねる)
    average_figures_path = os.path.join(experiment_root, "average_figures")
    all_scores = [all_stability_scores, all_expressivity_scores, all_compositionality_scores, all_alternation_scores, all_topsim_scores, all_posdis_scores]
    
    for scores, name in zip(all_scores, score_names):
        plot_average_results(scores, name, generations, average_figures_path)

    # 2. 意味-形式ペアのTXT保存 (rep_i/generations/ に保存)
    save_data_txt(all_meaning_signal_pairs_by_rep, experiment_root)
        
if __name__ == "__main__":
    main()

Experiment Root: /Users/iwamurairifuki/evolang2026/out/exp20251103_213554_gen50_m8_f7_alpha5

--- Replicates: 0 ---

--- Replicates: 1 ---

--- Replicates: 2 ---


/Users/iwamurairifuki/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))



--- Replicates: 3 ---

--- Replicates: 4 ---

--- Saving Final Results ---
Generation data saved to individual 'rep_i/generations' folders.
